# Feedforward Neural Network

#### Logtistic Regression Problems
<img src="https://user-images.githubusercontent.com/60699771/85823477-80251200-b7b8-11ea-8371-b1f947ac8a2b.png" align=left>

### 1.1 Introducing a Non-linear Function

<img src="https://user-images.githubusercontent.com/60699771/85823837-4b658a80-b7b9-11ea-817a-fd0e635061bd.png" align=left>

## 1.2 Non-linear Function In-Depth

* Function: takes a nuber & perform mathmetical operation
* Common Types of Non-linearity
    * ReLu(rectified Linear Units)
    * Sigmoid
    * Tanh
<br>
<img src="https://user-images.githubusercontent.com/60699771/85824643-41448b80-b7bb-11ea-80b3-6c573ddeac12.png" align=left>
<img src="https://user-images.githubusercontent.com/60699771/85824654-44d81280-b7bb-11ea-9729-17ed2379b062.png" align=left>

## 2. Feedforward Neural Network 설계

### 2.1 Model A : Hidden Layer FeedForward Neural Network (Sigmoid Activation)
<img src="https://user-images.githubusercontent.com/60699771/85845029-8f1ebb00-b7de-11ea-94d6-00544adec536.png" align=left>


### Steps
* Step 1 : Load Dataset
* Step 2 : Make Dataset Iterable
* Step 3 : Create Model Class
* Step 4 : Instantiate Model Class
* Step 5 : Instantiate Loss Class
* Step 6 : Instantiate Optimizer Class
* Step 7 : Train Model

### Step 1. Loading MNIST Train Dataset
* images from 1 to 9

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

In [2]:
train_dataset = dsets.MNIST(root='./data',
                           train= True,
                           transform= transforms.ToTensor(),
                           download=True)

test_dataset = dsets.MNIST(root='./data',
                          train=False,
                          transform = transforms.ToTensor())

### Step 2. Make Dataset Iterable

In [3]:
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)
print(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=False)

5


### Step 3. Create Model Class

In [4]:
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        
        # Non-linearity
        self.sigmoid = nn.Sigmoid()
        
        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        # Linear function
        out = self.fc1(x)
        
        # Non-linearity
        out = self.sigmoid(out)
        
        # Linear function(readout)
        out = self.fc2(out)
        
        return out

### Step 4. Instantiate Model Class
* Input dimension: 784
    * size of image : 2 x 28 =784
* Output dimension: 10
    * 0,1,2,3,4,5,6,7,8,9
* Hidden dimension: 100
    * Can be any number
    * Similar term
        * Number of neurons
        * Number of non-linear activation functions

In [5]:
input_dim = 28*28
hidden_dim = 100
output_dim = 10

model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

### Step 5. Instantiate Loss Class
* Feedforward Nerual Network : Cross Entropy Loss
    * cf : Linear Regression: MSE
    * cf : Logistic Regression : Cross Entropy Loss

In [6]:
criterion = nn.CrossEntropyLoss()

### Step 6. instantiate Optimizer Class
<img src="https://user-images.githubusercontent.com/60699771/85815174-b8215a80-b7a2-11ea-917f-de5c35eb9ea9.png" align=left>

In [7]:
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

#### Parameteres in Depth

In [8]:
print(model.parameters())

print(len(list(model.parameters())))

# Hidden Layer Parameters
print(list(model.parameters())[0].size())

# FC 1 Bias Parameters
print(list(model.parameters())[1].size())

# FC 2 Parameters
print(list(model.parameters())[2].size())

# FC 3 Bias Parameters
print(list(model.parameters())[3].size())

<generator object Module.parameters at 0x000001F81CB48EC8>
4
torch.Size([100, 784])
torch.Size([100])
torch.Size([10, 100])
torch.Size([10])


<img src="https://user-images.githubusercontent.com/60699771/85856617-4c1b1280-b7f3-11ea-8ee3-81be9203d0b0.png" align=left>

### Step7: Train a Model
* process
    1. Convert inputs/labels to variables
    2. Clear gradient buffets
    3. Get output given inputs
    4. Get loss
    5. Get gradients w.r.t parameters
    6. Update parameters using gradients
        * parameters = parameters - lr * parameters_gradients
    7. Repeat

In [9]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load Images as Variable
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        # Clear gradients w.r.t parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model.forward(images)
        
        # Calculate Loss: softmax -> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting Gradients w.r.t parameters
        loss.backward()
        
        #Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images to a Torch Variable
                images = Variable(images.view(-1, 28*28))
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()
                
            accuracy = 100 * correct /total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data, accuracy))

..\aten\src\ATen\native\BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


Iteration: 500. Loss: 0.8230674266815186. Accuracy: 86
Iteration: 1000. Loss: 0.4828628599643707. Accuracy: 89
Iteration: 1500. Loss: 0.2595706284046173. Accuracy: 90
Iteration: 2000. Loss: 0.23956598341464996. Accuracy: 91
Iteration: 2500. Loss: 0.3080233633518219. Accuracy: 91
Iteration: 3000. Loss: 0.19435890018939972. Accuracy: 91


### 2.2 Model B. Hidden Layer Feedforward Neural Network(Tanh Avtivation)

In [10]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

'''
STEP 1: LOADING DATASET
'''

train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

'''
STEP 2: MAKING DATASET ITERABLE
'''

batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

'''
STEP 3: CREATE MODEL CLASS
'''
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        
        # Non-linearity
        self.tanh = nn.Tanh()
        
        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        # Linear function
        out = self.fc1(x)
        
        # Non-linearity
        out = self.tanh(out)
        
        # Linear function(readout)
        out = self.fc2(out)
        
        return out
    
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 28*28
hidden_dim = 100
output_dim = 10

model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()


'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load Images as Variable
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        # Clear gradients w.r.t parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model.forward(images)
        
        # Calculate Loss: softmax -> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting Gradients w.r.t parameters
        loss.backward()
        
        #Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images to a Torch Variable
                images = Variable(images.view(-1, 28*28))
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()
                
            accuracy = 100 * correct /total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data, accuracy))

Iteration: 500. Loss: 0.3297162353992462. Accuracy: 91
Iteration: 1000. Loss: 0.3135600984096527. Accuracy: 92
Iteration: 1500. Loss: 0.20512685179710388. Accuracy: 93
Iteration: 2000. Loss: 0.14603491127490997. Accuracy: 94
Iteration: 2500. Loss: 0.2541802227497101. Accuracy: 94
Iteration: 3000. Loss: 0.2612457573413849. Accuracy: 95


### 2.3 Model C. Hidden Layer Feedforward Neural Network(ReLU Ativation)

In [11]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

'''
STEP 1: LOADING DATASET
'''

train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

'''
STEP 2: MAKING DATASET ITERABLE
'''

batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

'''
STEP 3: CREATE MODEL CLASS
'''
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        
        # Non-linearity
        self.relu = nn.ReLU()
        
        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        # Linear function
        out = self.fc1(x)
        
        # Non-linearity
        out = self.relu(out)
        
        # Linear function(readout)
        out = self.fc2(out)
        
        return out
    
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 28*28
hidden_dim = 100
output_dim = 10

model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()


'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load Images as Variable
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        # Clear gradients w.r.t parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model.forward(images)
        
        # Calculate Loss: softmax -> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting Gradients w.r.t parameters
        loss.backward()
        
        #Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images to a Torch Variable
                images = Variable(images.view(-1, 28*28))
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()
                
            accuracy = 100 * correct /total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data, accuracy))

Iteration: 500. Loss: 0.34208956360816956. Accuracy: 91
Iteration: 1000. Loss: 0.2666618824005127. Accuracy: 93
Iteration: 1500. Loss: 0.219783216714859. Accuracy: 93
Iteration: 2000. Loss: 0.16866302490234375. Accuracy: 94
Iteration: 2500. Loss: 0.19311000406742096. Accuracy: 95
Iteration: 3000. Loss: 0.17960584163665771. Accuracy: 95


### 2.4 Model D : 2 Hidden Layer Feedforward Nerual Network (ReLU Activation)
<img src="https://user-images.githubusercontent.com/60699771/85862216-6dccc780-b7fc-11ea-8b34-3fcba10e2607.png" align=left>

In [12]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

'''
STEP 1: LOADING DATASET
'''

train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

'''
STEP 2: MAKING DATASET ITERABLE
'''

batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

'''
STEP 3: CREATE MODEL CLASS
'''
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FeedforwardNeuralNetModel, self).__init__()
        
        # Linear function 1 : 784 -> 100
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        # Non-linearity 1
        self.relu1 = nn.ReLU()
        
        # Linear function 2 : 100  -> 100
        self.fc2 = nn.Linear(input_dim, hidden_dim)
        # Non-linearity 2
        self.relu2 = nn.ReLU()
        
        # Linear function (readout) : 100 -> 10
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        # Linear function 1
        out = self.fc1(x)        
        # Non-linearity 1
        out = self.relu1(out)
        
        # Linear function 2
        out = self.fc2(x)        
        # Non-linearity 2
        out = self.relu2(out)
        
        # Linear function 3(readout)
        out = self.fc3(out)
        
        return out        
    
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 28*28
hidden_dim = 100
output_dim = 10

model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()


'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load Images as Variable
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        # Clear gradients w.r.t parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model.forward(images)
        
        # Calculate Loss: softmax -> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting Gradients w.r.t parameters
        loss.backward()
        
        #Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images to a Torch Variable
                images = Variable(images.view(-1, 28*28))
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()
                
            accuracy = 100 * correct /total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data, accuracy))

Iteration: 500. Loss: 0.3146301507949829. Accuracy: 91
Iteration: 1000. Loss: 0.24152250587940216. Accuracy: 93
Iteration: 1500. Loss: 0.33517611026763916. Accuracy: 94
Iteration: 2000. Loss: 0.13933822512626648. Accuracy: 94
Iteration: 2500. Loss: 0.12824854254722595. Accuracy: 95
Iteration: 3000. Loss: 0.13613037765026093. Accuracy: 95


### 2.4 Model D : 3 Hidden Layer Feedforward Nerual Network (ReLU Activation)
<img src="https://user-images.githubusercontent.com/60699771/85863534-3f4fec00-b7fe-11ea-9834-e58cac414fbf.png" align=left>

In [13]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

'''
STEP 1: LOADING DATASET
'''

train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

'''
STEP 2: MAKING DATASET ITERABLE
'''

batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

'''
STEP 3: CREATE MODEL CLASS
'''
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FeedforwardNeuralNetModel, self).__init__()
        
        # Linear function 1 : 784 -> 100
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        # Non-linearity 1
        self.relu1 = nn.ReLU()
        
        # Linear function 2 : 100  -> 100
        self.fc2 = nn.Linear(input_dim, hidden_dim)
        # Non-linearity 2
        self.relu2 = nn.ReLU()
        
        # Linear function 3 : 100  -> 100
        self.fc3 = nn.Linear(input_dim, hidden_dim)
        # Non-linearity 3
        self.relu3 = nn.ReLU()
        
        # Linear function 4(readout) : 100 -> 10
        self.fc4 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        # Linear function 1
        out = self.fc1(x)        
        # Non-linearity 1
        out = self.relu1(out)
        
        # Linear function 2
        out = self.fc2(x)        
        # Non-linearity 2
        out = self.relu2(out)
        
        # Linear function 3
        out = self.fc3(x)        
        # Non-linearity 3
        out = self.relu3(out)
        
        # Linear function 4(readout)
        out = self.fc4(out)
        
        return out        
    
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 28*28
hidden_dim = 100
output_dim = 10

model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()


'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load Images as Variable
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        # Clear gradients w.r.t parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model.forward(images)
        
        # Calculate Loss: softmax -> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting Gradients w.r.t parameters
        loss.backward()
        
        #Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images to a Torch Variable
                images = Variable(images.view(-1, 28*28))
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()
                
            accuracy = 100 * correct /total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data, accuracy))

Iteration: 500. Loss: 0.33939915895462036. Accuracy: 91
Iteration: 1000. Loss: 0.4601387679576874. Accuracy: 92
Iteration: 1500. Loss: 0.20997871458530426. Accuracy: 94
Iteration: 2000. Loss: 0.13804380595684052. Accuracy: 94
Iteration: 2500. Loss: 0.22034579515457153. Accuracy: 95
Iteration: 3000. Loss: 0.13787129521369934. Accuracy: 95


### Deep Learning
* 2 Ways to expand a nerual network
    * more non-linear avtivation units(neurons)
    * More hidden layers
* Cons
    * Need a larger dataset
        * CUrse of dimensionality
    * Does not neccessarily higher accuarvy